# 12. Play with Connect 4

## Connect 4 (중력4목)

이전까지 오목을 학습하려고 이런저런 시도를 했지만
그리 좋은 결과를 얻지는 못했습니다.

여기서는 상태와 입력을 더 적은 Connect 4에
RL을 시도해봅니다.

## 이전까지의 문제점

강화학습을 할 때에는 최종적으로는 policy $\pi$를
탐색하는 것이 목표가 되며,
이 $\pi$를 직접 학습시키느냐 (policy-based)
아니면 value function인 $v$나 $q$를 학습시켜서
$\pi$를 구하느냐 (value-based)로 나뉘게 됩니다.

이전까지 시도했던 방법들은 $q$를 Monte Carlo
policy evaluation으로 value function을
근사를 하려고 했습니다.
처음에는 $q$를 근사하려고 했지만, 오목판의
상태와 행동이 너무 많아서 잘 안 되나 싶어서 $v$를
근사해보고자 하였고, 실제로는 둘 다 잘 되지 않았습니다.

다만, 이전 시도에는 몇가지
문제가 있는데,

- Policy를 $\pi = \text{argmax} \circ q$라고
두고 $q$를 갱신하다보니, policy가 unstable하게
변하는 듯 합니다.
- Value network의 complexity도,
학습이 어느 정도 진행될 것이라고 생각한 time bound도
너무 작았습니다.
Connect 4는 오목보다는 상태나 행동이 적은 편인데,
이 게임도 7개의 layer로 하루종일 학습시켜봐야
value network만으로 사람을 이기기는 쉽지 않다고 합니다.

그래서 여기서는 기존 오목 대신에 Connect4 (중력4목)에
기존보다 complexity를 높인 neural network를 사용하여
value-based learning과 policy-based learning을
적용해보고 결과를 확인해봅니다.

## 환경설정

여기서는 Mock4.py를 사용합니다.

https://github.com/lumiknit/mock4.py

In [1]:
!rm -rf mock4.py m4
!git clone https://github.com/lumiknit/mock4.py.git
!mv mock4.py m4
!mv m4/mock4.py .
from mock4 import *

Cloning into 'mock4.py'...
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 13 (delta 3), reused 6 (delta 1), pack-reused 0
Unpacking objects: 100% (13/13), done.


Mock5.py와 거의 비슷하게 사용하면 됩니다.

In [2]:
Mock4().play(agent_greedy, agent_greedy, p_msg=False)

-----------------
[ Turn  29 ; 2P ]
| 0 1 2 3 4 5 6 |
| O . O X X . . |
| X O O X X . . |
| O O O X X . . |
| O X X O O . . |
| X O O X X . . |
| X O O O X . . |
1P Win (<function agent_greedy at 0x7f9ce4cfb710>)


2

In [3]:
test_mock4(100, agent_random, agent_greedy)

** Test
* A1 = <function agent_random at 0x7f9ce4cfb0e0>
* A2 = <function agent_greedy at 0x7f9ce4cfb710>
Total = 100 games
W1 0 (0.000) / Dr 0 (0.000) / W2 100 (1.000)


외에 pytorch, numpy를 불러옵니다.

In [4]:
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: {}".format(device))

Device: cuda


In [5]:
class Flatten(nn.Module):
  def forward(self, x):
    if len(x.shape) == 3: return x.view(-1)
    else: return x.flatten(1, -1)

In [6]:
class Replay():
  def __init__(self, size):
    self.size = size
    self.b = []

  def remove_olds(self):
    if len(self.b) > self.size:
      self.b = self.b[-self.size :]
  
  def add(self, S0, A0, R0, S1):
    self.b.append((S0, A0, R0, S1))
    self.remove_olds()
  
  def sample(self, size):
    Z = [None] * size
    for i in range(size):
      j = np.random.randint(len(self.b))
      Z[i] = self.b[j]
    S0 = [z[0] for z in Z]
    A0s = [z[1] for z in Z]
    R0 = [z[2] for z in Z]
    S1 = [z[3] for z in Z]
    return S0, A0s, R0, S1

In [7]:
# Scope, to separate namespace
class Scope(): pass

## DQN

Deep neural network를 이용해서
Q-Learning을 합니다.

- Action value function $q$를
deep neural network로 구성합니다.
- Policy $\pi$는 $q$를 그대로 쓰되,
action을 선택할 떄 argmax로 선택합니다.
(Softmax로 확률처럼 바꿀 수는 있습니다만..)
Policy improvement는 특정 개수의 episode가 
진행되면
$\pi$를 $q$로 바꾸는 식으로 이루어집니다.
- $\alpha$, $\epsilon$을 모두 decay합니다.
- Batch normalization을 사용합니다.
- Episode를 진행하며 replay memory를 누적시키고
replay memory에서 샘플링한 batch로 학습시킵니다.

In [ ]:
ql = Scope()

In [ ]:
## nn
def _new_nn():
  W = 7
  H = 6
  net = nn.Sequential(
      # 01
      nn.Conv2d(3, 64, 3, padding='same'),
      nn.BatchNorm2d(64),
      nn.ReLU(),
      # 02
      nn.Conv2d(64, 64, 3, padding='same'),
      nn.BatchNorm2d(64),
      nn.MaxPool2d(2),
      nn.ReLU(),
      # 03
      nn.Conv2d(64, 32, 3, padding='same'),
      nn.BatchNorm2d(32),
      nn.ReLU(),
      # 04
      nn.Conv2d(32, 32, 3, padding='same'),
      nn.BatchNorm2d(32),
      nn.ReLU(),
      # 05
      nn.Conv2d(32, 8, 3, padding='same'),
      nn.BatchNorm2d(8),
      nn.ReLU(),
      # Lin01
      Flatten(),
      nn.Linear(8 * (W // 2) * (H // 2), 20),
      nn.BatchNorm1d(20),
      nn.ReLU(),
      # Lin02
      nn.Linear(20, W)
  ).to(device)
  return net
ql.new_nn = _new_nn

def _update_policy(policy, q):
  policy.load_state_dict(q.state_dict())
  q.train()
  policy.eval()
ql.update_policy = _update_policy

def _init_nn():
  ql.policy = ql.new_nn()
  ql.q = ql.new_nn()
  ql.update_policy(ql.policy, ql.q)
ql.init_nn = _init_nn

In [ ]:
# Policy = epsilon-greedy for q
def _agent_policy(epsilon):
  def agent(game):
    if np.random.uniform() < epsilon: return agent_random(game)
    X = game.tensor().unsqueeze(dim=0).to(device)
    M = game.tensor_full()
    with torch.no_grad():
      Q = ql.policy(X)
      Q = Q.squeeze(dim=0)
    Q[M] = -float('inf')
    A = torch.argmax(Q)
    return A
  return agent
ql.agent_policy = _agent_policy

In [ ]:
ql.replay = Replay(65536)
ql.replay_terminal = Replay(2048)

In [ ]:
# Q-Learning
def _learn(
    opt,
    loss_fn,
    n_episode,
    n_epoch,
    int_policy_update,
    gamma,
    alpha_fn,
    epsilon_fn,
    sz_sample,
    sz_sample_terminal
):
  init_left = 20
  epi = 0
  while epi < n_episode:
    # -- Get parameters
    alpha = alpha_fn(epi)
    epsilon = epsilon_fn(epi)
    # -- Run Game
    game = Mock4()
    a1 = ql.agent_policy(epsilon)
    a2 = agent_greedy
    result = game.play(a1, a2, p_msg=False, p_res=False)
    reward = 1
    if result == 0: # Draw
      reward = 0
      result = 1
    # -- Append to Replay
    S1_p, S1_o = None, None
    while len(game.history) > 0:
      a = int(game.undo() / game.h)
      S0_p = game.tensor(player=result)
      S0_o = game.tensor(player=(3 - result))
      ql.replay.add(S0_p, a, reward, S1_p)
      ql.replay.add(S0_o, a, -reward, S1_o)
      if S1_p is None:
        ql.replay_terminal.add(S0_p, a, reward, S1_p)
        ql.replay_terminal.add(S0_o, a, -reward, S1_o)
      S1_p, S1_o = S0_p, S0_o
      reward = 0
    # -- Sampling and learning
    if len(ql.replay.b) >= sz_sample:
      S_0, As, Rs, S_1 = ql.replay.sample(sz_sample)
      # Append Terminal states
      t_S_0, t_As, t_Rs, t_S_1 = ql.replay_terminal.sample(sz_sample_terminal)
      S_0 += t_S_0
      As += t_As
      Rs += t_Rs
      S_1  += t_S_1
      # Tensor-fy
      X_0 = torch.stack(S_0).to(device)
      R = torch.tensor(Rs, dtype=torch.float).to(device)
      Sz_1 = [torch.zeros(3, game.w, game.h) if s is None else s for s in S_1]
      Snone_1 = [s is None for s in S_1]
      X_1 = torch.stack(Sz_1).to(device)
      # Calc Curr Q
      with torch.no_grad():
        Q_0 = ql.q(X_0)
        Q_1 = ql.q(X_1)
      Qa_0 = Q_0[range(len(As)), As]
      Qmax_1 = torch.max(Q_1, dim=1).values
      # Q_0 <- Q_0 + alpha * (R + gamma * max Q_1 - Q_0) if not terminated
      Qtgt_0 = Qa_0 + alpha * (R + gamma * Qmax_1 - Qa_0)
      # Q_0 <- R otherwise
      Qtgt_0[Snone_1] = R[Snone_1]
      # Clip
      Qtgt_0[Qtgt_0 > 1.0] = 1.0
      Qtgt_0[Qtgt_0 < -1.0] = -1.0
      if init_left > 0:
        init_left -= 1
        Qtgt_0 = torch.zeros(Qtgt_0.shape).to(device)
        print("INIT")
      # Learn
      loss_list = []
      for e in range(n_epoch):
        opt.zero_grad()
        Q_0 = ql.q(X_0)
        Qa_0 = Q_0[range(len(As)), As]
        loss = loss_fn(Qa_0, Qtgt_0)
        loss_list.append(loss.mean().item())
        loss.backward()
        opt.step()
      epi += 1
      print("Ep #{} (#Repl={}) Loss {:.8f}α -> {:.8f}α".format(
          epi, len(ql.replay.b), loss_list[0] / alpha, loss_list[-1] / alpha))
      # Update Policy
      if (epi + 1) % int_policy_update == 0:
        ql.update_policy(ql.policy, ql.q)
    else: print("Accumulating Replay... (#={})".format(len(ql.replay.b)))
ql.learn = _learn

In [ ]:
def run():
  ql.init_nn()
  opt = optim.Adam(ql.q.parameters(), lr=1e-3, weight_decay=1e-6)
  loss_fn = nn.SmoothL1Loss()
  n_episode = 1000
  n_epoch = 10
  int_policy_update = 10
  alpha_fn = lambda n: 1
  gamma = 0.99
  epsilon_fn = lambda n: max(0.05, 0.3 * (0.99 ** n))
  sz_sample_terminal = 256
  sz_sample = 4096 - sz_sample_terminal

  ql.learn(
      opt=opt,
      loss_fn=loss_fn,
      n_episode=n_episode,
      n_epoch=n_epoch,
      int_policy_update=int_policy_update,
      alpha_fn=alpha_fn,
      gamma=gamma,
      epsilon_fn=epsilon_fn,
      sz_sample=sz_sample,
      sz_sample_terminal=sz_sample_terminal
  )
run()

In [ ]:
test_mock4(100, agent_random, ql.agent_policy(0))
test_mock4(100, agent_greedy, ql.agent_policy(0))

** Test
* A1 = <function agent_random at 0x7fc6a3aaf0e0>
* A2 = <function _agent_policy.<locals>.agent at 0x7fc5e0a5f0e0>
Total = 100 games
W1 48 (0.480) / Dr 0 (0.000) / W2 52 (0.520)
** Test
* A1 = <function agent_greedy at 0x7fc6a3aaf710>
* A2 = <function _agent_policy.<locals>.agent at 0x7fc5e0a5fe60>
Total = 100 games
W1 100 (1.000) / Dr 0 (0.000) / W2 0 (0.000)


### 학습 방법

- 학습은 처음에 약 $20000$개의 중간상태
($< 5000$개의 기보)를 만들어둔 상태에서
매 episode를 진행하면서 이전 replay를 가져와
학습하도록 하였습니다.
- 상대 agent로는 현재 이기기 위한 목표인
`agent_greedy` (analysis-based)를 사용했습니다.
- 한번에 $4096$개의 중간상태 ($\simeq 100$개의 기보)
를 batch로 가져와서 $q$를 근사하며,
이를 1000번 반복합니다.

### 결과

- `agent_random` 상대로 약 50%의 승률을 얻고,
`agent_greedy`에게는 완패합니다.
- 학습이 진행되면 $q$가 특정 값으로 수렴해야 하는데,
$\alpha$를 어떻게 decay 하는지와 무관하게 잘 수렴하지
않습니다.
- 오목이나 사목 같은 경우에는 episode가 충분히 짧고
중간 행동에 대한 reward가 없기 때문에 $q$를 이용해서
근사하는 것보다는 Monte Carlo 방식으로 샘플을
누적시키는 것이 훨씬 빠르게 수렴하는 것 같습니다.
아마 위 방식도 훨씬 긴 시간동안 실행하면 수렴할 수도
있을 것 같지만, 위 학습을 실행하는데 이미
30분가량 소모되기 때문에 중단하였습니다.

## Monte Carlo Policy Gradient

REINFORCE 알고리즘을 사용합니다.

- Policy $\pi$에 대한 neural network를 구성합니다.
network structure는 위 DQN에서 사용한 $q$와
완전히 동일하되, batch normalization만 제외하며,
출력에 softmax를 사용합니다.
- Policy가 indicator function이 아니기 때문에
행동은 확률적으로 시행됩니다.
애초에 $q$가 없기도 하고, 확률적으로 시행하기 때문에
자동으로 탐색을 하므로, $\epsilon$ 같은
매개변수는 없습니다.
- Gradient ascent에 torch의 optimizer를 사용합니다.
- Reward는 게임이 끝난 시점에서만 주어지므로,
return $G$는 게임 결과의 상수배로 나타납니다.
기존에는 이 $G$가 이겼으면 $G_t = \gamma^{T-t} > 0$,
지면 $G_t = - \gamma^{T-t} < 0$, 무승부면 $G = 0$으로 주었는데, 
여기서는 $- G \log \pi$, 즉 $\pi$의 negative loss
에 return을 곱한 것을 loss로 사용하기 때문에,
return $G$가 음수라면 weight exploding이
발생합니다.
따라서, 승패 보상을 $[0, 1]$로 조절합니다.
- 추가적으로, 여기서는 NLL Loss 같은 loss function
을 사용하는데, NLL Loss는 label
(여기서는 return $G$)가 0이면 loss가 0이기 때문에
학습에 영향을 끼치지 않습니다.
즉, 이런 경우인 패배하는 경우는 여기서는 학습 데이터로
사용하지 않습니다.


In [8]:
mcpg = Scope()

In [109]:
## nn
def _new_nn():
  W = 7
  H = 6
  net = nn.Sequential(
      # 01
      nn.Conv2d(3, 64, 3, padding='same'),
      nn.ReLU(),
      # 02
      nn.Conv2d(64, 64, 3, padding='same'),
      nn.MaxPool2d(2),
      nn.ReLU(),
      # 03
      nn.Conv2d(64, 32, 3, padding='same'),
      nn.ReLU(),
      # 04
      nn.Conv2d(32, 32, 3, padding='same'),
      nn.ReLU(),
      # 05
      nn.Conv2d(32, 8, 3, padding='same'),
      nn.ReLU(),
      # Lin01
      Flatten(),
      nn.Linear(8 * (W // 2) * (H // 2), 20),
      nn.ReLU(),
      # Lin02
      nn.Linear(20, W),
      # Softmax
      nn.Softmax(dim=-1)
  ).to(device)
  return net
mcpg.new_nn = _new_nn

def _init_nn():
  mcpg.policy = mcpg.new_nn()
mcpg.init_nn = _init_nn

In [75]:
def _argrnd(tensor):
  # Choosed argument randomly by p, the probablity of each arguments
  # e.g. if p=[0.2, 0.5, 0.3], choose 0 in 20%, choose 1 in 50%, choose 2 in 30%
  if len(tensor.shape) == 1:
    p = tensor.to('cpu').numpy()
    p = p / p.sum()
    return torch.tensor(np.random.choice(tensor.shape[0], p=p))
  elif len(tensor.shape) == 2:
    a = []
    for i in range(tensor.shape[0]):
      p = tensor[i, :].to('cpu').numpy()
      p = p / p.sum()
      a.append(np.random.choice(tensor.shape[1], p=p))
    return torch.tensor(a)
  else: raise ValueError("Unsupported Shape {} for argrnd".format(tensor.shape))
mcpg.argrnd = _argrnd

In [76]:
# Policy = epsilon-greedy for policy
def _agent_policy(epsilon):
  def agent(game):
    if np.random.uniform() < epsilon: return agent_random(game)
    X = game.tensor().unsqueeze(dim=0).to(device)
    M = game.tensor_full()
    with torch.no_grad():
      Q = mcpg.policy(X)
      Q = Q.squeeze(dim=0)
    Q[M] = 0.0
    A = mcpg.argrnd(Q).item()
    return A
  return agent
mcpg.agent_policy = _agent_policy

In [68]:
def _loss_fn(policy, v):
  # Original GD (minimization) : theta <- theta - alpha D J(theta)
  # In policy gradient (maximization) : theta <- theta + alpha v D log pi
  # => Negative log likelihood weighted by reward v
  return - (v * torch.log(policy)).mean()
mcpg.loss_fn = _loss_fn

In [164]:
# REINFORCE
def _learn(
    opt,
    n_episode,
    n_epoch,
    gamma,
    batch_size
):
  epi = 0
  Xs, As, Vs = [], [], []
  while epi < n_episode:
    # -- Run Game
    game = Mock4()
    a1 = mcpg.agent_policy(0)
    a2 = agent_greedy
    result = game.play(a1, a2, p_msg=False, p_res=False)
    reward = 1
    if result == 0: # Draw
      reward = 0.5
      result = 1
    # -- Append to Replay
    while len(game.history) > 0:
      a = int(game.undo() / game.h)
      S0_p = game.tensor(player=result)
      Xs.append(S0_p)
      As.append(a)
      Vs.append(reward)
      reward *= gamma
    if len(Xs) >= batch_size:
      # Tensor-fy
      X = torch.stack(Xs).to(device)
      A = torch.tensor(As).unsqueeze(dim=1).to(device)
      V = torch.tensor(Vs, dtype=torch.float).to(device)
      # Learn
      loss_list = []
      for e in range(n_epoch):
        opt.zero_grad()
        pi_s = mcpg.policy(X)
        pi_sa = pi_s.gather(1, A).squeeze(dim=1)
        loss = mcpg.loss_fn(pi_sa, V)
        loss_list.append(loss.mean().item())
        loss.backward()
        opt.step()
      if epi % 100 == 0:
        print("Ep #{:<6d} Loss {:13.10f} -> {:13.10f}".format(
          epi, loss_list[0], loss_list[-1]))
      Xs, As, Vs = [], [], []
      epi += 1
mcpg.learn = _learn

In [ ]:
def run():
  mcpg.init_nn()

  opt = optim.Adam(mcpg.policy.parameters(), lr=1e-2, weight_decay=1e-5)
  n_episode = 20000
  n_epoch = 2
  gamma = 0.99
  batch_size = 50

  mcpg.learn(
      opt=opt,
      n_episode=n_episode,
      n_epoch=n_epoch,
      gamma=gamma,
      batch_size=batch_size
  )
run()

In [172]:
test_mock4(1000, agent_random, mcpg.agent_policy(0))
test_mock4(1000, agent_greedy, mcpg.agent_policy(0))

** Test
* A1 = <function agent_random at 0x7f9ce4cfb0e0>
* A2 = <function _agent_policy.<locals>.agent at 0x7f9c08c4a5f0>
Total = 1000 games
W1 302 (0.302) / Dr 1 (0.001) / W2 697 (0.697)
** Test
* A1 = <function agent_greedy at 0x7f9ce4cfb710>
* A2 = <function _agent_policy.<locals>.agent at 0x7f9c08c4a050>
Total = 1000 games
W1 995 (0.995) / Dr 0 (0.000) / W2 5 (0.005)


### 학습방법

- 원래 알고리즘이 각 episode 별로 gradient ascent
를 시행하기 때문에, 여기서 batch는
약 2개의 episode가 들어갈 정도인 50을 사용합니다.
- 하나의 batch에 대해 gradient ascent는 2번만
합니다.
(원래 REINFORCE는 하나의 episode에 대해 한번씩만
ascent를 하는데, loss의 변화를 보기 위해 2번 합니다.)
- Episode의 수는 학습시간이 위의 DQN의
학습시간인 30분과 비슷하게 맞추기
위해 $40000$
(size 50의 batch로 20000번 학습하는데,
batch당 약 2개의 episode가 들어가므로)
개의 게임을 진행합니다.
- 상대 agent로는 `agent_greedy`를 사용합니다.

### 결과

- `agent_random`을 상대로는 약 70%,
`agent_greedy`를 상대로는 약 0.5%의 승률을 보입니다.
- Policy가 확률적으로 동작하기 때문에, 실행을 할 때마다
결과는 달라지며, Softmax function with temperature
$S(\textbf{x}; \tau) = \text{Softmax}(\tau^{-1}\textbf{x})$를 사용하면
$\pi$에서 낮은 확률을 갖는 경우를 더 쳐내고, 높은
확률을 가지는 경우만을 골라내는 것도 가능합니다.
- 지금까지의 시도 중 처음으로 `agent_greedy`를
이기는 경우가 보입니다.
아직 승률이 매우 낮지만,
어쨌든 Connect4도 최대 7가지의 경우의 수를 최소
4번을 적절히 골라야 이길 수 있기 때문에,
$\pi$가 최선/차선의 선택 정도는 학습했다고 봐도
괜찮을 것 같습니다.

## 결론

- 지금까지의 시도들 중 policy graident는 처음으로,
그리고 유일하게 analysis-based greedy algorithm을
상대로 승리를 보여주었습니다.
- Policy gradient로 이길 수 있었던 것이 위 DQN에서는
$\epsilon$-greedy로 탐색을 하는데 반해
policy gradient는 $\pi$ 자체를 일종의 확률 분포로
탐색을 하다보니 이기는 행동을 더 빠르게 탐색할 수 있지
않았나 싶습니다.
$q$에도 $\epsilon$-greedy가 아니라 softmax를 하여
policy를 추출하는 것은 가능한데, 이 경우에도
비슷한 결과를 얻을 수 있을지는 의문입니다.
(다만, DQN 방식대로 replay memory로 학습하는
것은 현재로서는 너무 오래 걸리다보니
나중에 시험해보기로 합니다.)
- 이제 policy gradient를 통해
승리할 수 있는 수가 $\pi$에서 비교적 높은 확률로
나타나므로, 이를 이용해서 tree search를 해보도록
합니다.